In [4]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_8d0310c12cd742c78d2bebe215924141 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='oBVdaoaAXRjIzRY3FFDHNIT8pVCv3FXg_Nz9ZGqpYsbm',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.eu.cloud-object-storage.appdomain.cloud')

streaming_body_1 = client_8d0310c12cd742c78d2bebe215924141.get_object(Bucket='alplantdiseaseclassification-donotdelete-pr-e0fuqvoge2arv0', Key='Fertilizers_Recommendation_ System_For_Disease_ Prediction.zip')['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [5]:
ls

In [6]:
from io import BytesIO
import zipfile
unzip=zipfile.ZipFile(BytesIO(streaming_body_1.read()),'r')
file_paths=unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [7]:
ls

'Dataset Plant Disease'/


In [8]:
pwd

'/home/wsuser/work'

In [9]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [10]:
train_datagen=ImageDataGenerator(rescale=1./255,zoom_range=0.2,horizontal_flip=True,vertical_flip=False)

In [11]:
test_datagen=ImageDataGenerator(rescale=1./255)

In [12]:
ls

'Dataset Plant Disease'/


In [13]:
pwd

'/home/wsuser/work'

In [14]:
x_train=train_datagen.flow_from_directory(r"/home/wsuser/work/Dataset Plant Disease/fruit-dataset/fruit-dataset/train",target_size=(128,128),
                                        class_mode='categorical',batch_size=24)

Found 5384 images belonging to 6 classes.


In [15]:
x_test=test_datagen.flow_from_directory(r"/home/wsuser/work/Dataset Plant Disease/fruit-dataset/fruit-dataset/test",target_size=(128,128),
                                        class_mode='categorical',batch_size=24)

Found 1686 images belonging to 6 classes.


In [16]:
x_train.class_indices

{'Apple___Black_rot': 0,
 'Apple___healthy': 1,
 'Corn_(maize)___Northern_Leaf_Blight': 2,
 'Corn_(maize)___healthy': 3,
 'Peach___Bacterial_spot': 4,
 'Peach___healthy': 5}

In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Convolution2D,MaxPooling2D,Flatten

In [18]:
model=Sequential()

In [19]:
model.add(Convolution2D(32,(3,3),input_shape=(128,128,3),activation='relu'))

In [20]:
model.add(MaxPooling2D(pool_size=(2,2)))

In [21]:
model.add(Flatten())

In [22]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 126, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 63, 63, 32)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 127008)            0         
                                                                 
Total params: 896
Trainable params: 896
Non-trainable params: 0
_________________________________________________________________


In [23]:
model.add(Dense(300,activation='relu'))
model.add(Dense(150,activation='relu'))

In [24]:
model.add(Dense(6,activation='softmax'))

In [25]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [26]:
len(x_train)

225

In [28]:
model.fit_generator(x_train,steps_per_epoch=len(x_train),validation_data=x_test,validation_steps=len(x_test),epochs=10)

/tmp/wsuser/ipykernel_164/1582812018.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(x_train,steps_per_epoch=len(x_train),validation_data=x_test,validation_steps=len(x_test),epochs=10)


Epoch 1/10
225/225 [==============================] - 108s 479ms/step - loss: 0.9721 - accuracy: 0.8016 - val_loss: 0.2596 - val_accuracy: 0.9176
Epoch 2/10
225/225 [==============================] - 107s 474ms/step - loss: 0.2253 - accuracy: 0.9214 - val_loss: 0.2600 - val_accuracy: 0.9146
Epoch 3/10
225/225 [==============================] - 103s 456ms/step - loss: 0.1797 - accuracy: 0.9382 - val_loss: 0.1351 - val_accuracy: 0.9484
Epoch 4/10
225/225 [==============================] - 105s 468ms/step - loss: 0.1333 - accuracy: 0.9532 - val_loss: 0.0863 - val_accuracy: 0.9698
Epoch 5/10
225/225 [==============================] - 106s 469ms/step - loss: 0.1127 - accuracy: 0.9610 - val_loss: 0.1421 - val_accuracy: 0.9508
Epoch 6/10
225/225 [==============================] - 106s 470ms/step - loss: 0.1495 - accuracy: 0.9493 - val_loss: 0.0986 - val_accuracy: 0.9638
Epoch 7/10
225/225 [==============================] - 107s 475ms/step - loss: 0.1069 - accuracy: 0.9591 - val_loss: 0.1183 -

In [29]:
model.save('fruit_pred_K.h5')